### 종목 선택  
: 장 마감시에 이동평균선 간격이 좁은 종목들을 찾고, 다음 날 바로 매매 가능하도록  
: 이동평균선 간격이 좁은 종목 select

<br/>

> 모든 종목을 살펴서 그 중 선이 모이는 종목을 골라야 하므로 모든 etf 종목을 불러와야 함

In [1]:
import datetime
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup
import FinanceDataReader as fdr
from multiprocessing import Process

In [2]:
url = "https://kr.investing.com/etfs/south-korea-etfs"    # investing.com에서 국내 etf 종목 코드 가져올 것
user_agent_fox84 = 'Mozilla/5.0 (X11; Ubuntu 20.04; Linux arm; rv:99.0) Gecko/20100101 Firefox/99.0'
user_agent_fox99 = 'Mozilla/5.0 (X11; Ubuntu 20.04; Linux arm; rv:99.0) Gecko/20100101 Firefox/99.0'
user_agent_chrome = 'Mozilla/5.0 (X11; Ubuntu 20.10; Linux x86_64) AppleWebKit/537.36.0 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36.0'

headers = {'User-Agent':user_agent_fox84}
req = request.Request(url, data=None, headers=headers)
fp = request.urlopen(req)
source = fp.read()
fp.close()

html = BeautifulSoup(source, "html.parser")

symbol_source = html.find_all('td', attrs={'class':'left symbol'})

symbol_list = list()
for sym in symbol_source:
    symbol_list.append(sym.get_text())    # 전체 etf 종목 코드 가져오기

symbol_to_remove = list()    # 종목 코드 중 한국투자증권 api를 통해 구매를 시도할 때 오류가 나는 코드(etn 종목 코드)를 모을 list
for sym in symbol_list:
    for i in sym:
        if i.isalpha():    # etn 종목 코드는 알파벳이 있는 코드이므로 종목 코드 내에 알파벳이 있는지 확인
            symbol_to_remove.append(sym)    # 알파벳이 있는 종목 코드는 제거할 종목 리스트에 추가
for sym in symbol_to_remove:    # 제거할 종목 리스트를 순회하며 etn 종목이 종목 리스트에 있으면 종목 리스트에서 해당 종목 제거
    if sym in symbol_list:
        symbol_list.remove(sym)

> 종목별로 각 종목의 서로 다른 기간의 이동 평균선(ex. 5일선, 20일선, 120일선) 구하기

In [4]:
def get_data(code: str='252670'):
    time_now = datetime.datetime.now()
    period = time_now-datetime.timedelta(days=200)
    period = period.strftime('%Y-%m-%d')
    data = fdr.DataReader(code, period)

    return data

def get_distance_of_MovingAverage(code: str="005930", period: list=[5, 20, 120], col: str="Close", distance: float=10.0):
    data = get_data(code)
    
    ma = list()
    for i in range(3):
        data['ma_'+str(period[i])] = data[col].rolling(period[i]).mean()
        ma.append(float(data['ma_'+str(period[i])][-1]))
    
    f_symbol = open("/workspaces/codespaces-blank/project/stock/national/yj/data/symbol.txt", "a")
    if abs(ma[0]-ma[1])<=distance and abs(ma[1]-ma[2])<=distance:
        f_symbol.write(code+"\n")
    f_symbol.close()

> 이동 평균선 간의 간격이 작은 종목들 고르기

In [5]:
procs = []
for sym in symbol_list:
    proc = Process(target=get_distance_of_MovingAverage, args=(sym, [5, 20, 120], "Close", 50.0))
    procs.append(proc)
    proc.start()
for proc in procs:
    proc.join()

Process Process-37:
Process Process-63:
Process Process-6:
Process Process-40:
Process Process-36:
Process Process-25:
Process Process-32:
Process Process-18:
Process Process-69:
Process Process-35:
Traceback (most recent call last):
Process Process-65:
Process Process-68:
Process Process-3:
Process Process-47:
Traceback (most recent call last):
Process Process-59:
Process Process-19:
Process Process-57:
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-4:
Traceback (most recent call last):
  File "/home/codespace/miniconda3/envs/pr_stock/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/codespace/miniconda3/envs/pr_stock/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-20:
Traceback (most recent call last):
  File "/home/codespace/miniconda3/envs/pr_stock/lib/python3.8/multiprocessing/pr

거래량이 아예 없는 종목도 꽤 있음  
&rarr; 거래량이 없는 종목들을 걸러야 할 필요도 있음  

<br/>

> 거래량이 없는 종목들 거르기

In [37]:
import requests
import json
import yaml
import time

def get_access_token():
    """토큰 발급"""
    headers = {"content-type":"application/json"}
    body = {"grant_type":"client_credentials",
    "appkey":APP_KEY, 
    "appsecret":APP_SECRET}
    PATH = "oauth2/tokenP"
    URL = f"{URL_BASE}/{PATH}"
    res = requests.post(URL, headers=headers, data=json.dumps(body))
    ACCESS_TOKEN = res.json()["access_token"]
    return ACCESS_TOKEN

config_path_gcs = '/workspaces/codespaces-blank/project/stock/config.yaml'

with open(config_path_gcs, encoding='UTF-8') as f:
    _cfg = yaml.load(f, Loader=yaml.FullLoader)
APP_KEY = _cfg['APP_KEY']
APP_SECRET = _cfg['APP_SECRET']
ACCESS_TOKEN = ""
CANO = _cfg['CANO']
ACNT_PRDT_CD = _cfg['ACNT_PRDT_CD']
DISCORD_WEBHOOK_URL = _cfg['DISCORD_WEBHOOK_URL']
URL_BASE = _cfg['URL_BASE']

ACCESS_TOKEN = get_access_token()

In [38]:
def get_acml_volume(code: str="005930"):    # 일간 누적 거래량
    PATH = "uapi/domestic-stock/v1/quotations/inquire-price"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
            "authorization": f"Bearer {ACCESS_TOKEN}",
            "appKey":APP_KEY,
            "appSecret":APP_SECRET,
            "tr_id":"FHKST01010100"}
    params = {
    "fid_cond_mrkt_div_code":"J",
    "fid_input_iscd":code,
    }
    res = requests.get(URL, headers=headers, params=params)

    return int(res.json()['output']['acml_vol'])

In [15]:
print(get_acml_volume("137930"))

0


In [39]:
def get_trash_symbol():
    f_symbol = open("/workspaces/codespaces-blank/project/stock/national/yj/data/symbol.txt", "r")
    symbol_list = f_symbol.readlines()
    f_symbol.close()
    for i in range(len(symbol_list)):
        symbol_list[i] = symbol_list[i].strip()
    
    trash_list = list()
    for symbol in symbol_list:
        if get_acml_volume(symbol) == 0:
            trash_list.append(symbol)
        else:
            continue
    
    return trash_list
    

In [40]:
trash_list = get_trash_symbol()
f_symbol = open("/workspaces/codespaces-blank/project/stock/national/yj/data/symbol.txt", "r")
symbol_list = f_symbol.readlines()
f_symbol.close()
for i in range(len(symbol_list)):
        symbol_list[i] = symbol_list[i].strip()

for trash in trash_list:
    if trash in symbol_list:
          symbol_list.remove(trash)

f_symbol = open("/workspaces/codespaces-blank/project/stock/national/yj/data/symbol.txt", "w")
for symbol in symbol_list:
      f_symbol.write(symbol+"\n")
f_symbol.close()